# 📚 Tupi2LLM - Carregamento de Modelo com LoRA (QLoRA)
# -----------------------------------------------------
# Autor: Peterson
# Descrição: Este notebook carrega um modelo base com quantização em 4 bits
# (QLoRA) e aplica adaptadores LoRA personalizados (peterson047/Tupi2LLM)
# para realizar inferência com menor uso de memória.
#
# Requisitos:
# - GPU com suporte a bfloat16 (ex: T4, A100, etc.)
# - Transformers, PEFT, bitsandbytes instalados
# - Hugging Face token autenticado, se necessário
#
# 🚀 Etapas:
# 1. Instalação de dependências
# 2. Carregamento do tokenizer
# 3. Configuração da quantização
# 4. Carregamento do modelo base
# 5. Aplicação dos adaptadores LoRA
# 6. Pronto para inferência!
#
# 🧠 BASE_MODEL_ID: Ex: "meta-llama/Llama-2-7b-hf"
# 🪄 ADAPTER_HUB_ID: Ex: "peterson047/Tupi2LLM"


##Verificando GPU - (Ative a GPU no ambiente)

In [ ]:
!nvidia-smi

Fri Apr 18 03:17:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Instalando a biblioteca peft

In [ ]:
!pip install -U peft


##Instalando outras Dependências

In [ ]:
!pip install --upgrade pip
# Instala PyTorch compatível com a GPU do Colab (geralmente CUDA 12.1 é uma boa escolha recente)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# Instala as bibliotecas do Hugging Face e outras necessárias
!pip install transformers datasets accelerate peft bitsandbytes trl sentencepiece
# Instala scikit-learn e pandas para a divisão (se precisar refazer) e validação
!pip install scikit-learn pandas

Looking in indexes: https://download.pytorch.org/whl/cu121


##Login no Hugging Face Hub

In [ ]:
# Login no Hugging Face Hub para baixar modelos restritos
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

###Carregando Tokenizer


In [ ]:
from transformers import AutoTokenizer

BASE_MODEL_ID = "google/gemma-2b-it"

print(f"Carregando tokenizer de {BASE_MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

# Garante que há token de padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


Carregando tokenizer de google/gemma-2b-it...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


###Carregando modelo base

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL = "google/gemma-2b"
ADAPTER_HUB_ID = "peterson047/Tupi2LLM"
SUBFOLDER = "tupi-gemma-2b-lora-v8"

print("Carregando modelo base...")
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")

print(f"Carregando LoRA de {ADAPTER_HUB_ID}/{SUBFOLDER}...")
model = PeftModel.from_pretrained(
    model, ADAPTER_HUB_ID, subfolder=SUBFOLDER, is_trainable=False
)

print("Carregando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token


Carregando modelo base...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Carregando LoRA de peterson047/Tupi2LLM/tupi-gemma-2b-lora-v8...
Carregando tokenizer...


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### Definindo os parâmetros do modelo

In [ ]:
import torch

# --- Variáveis de Configuração ---
BASE_MODEL_ID = "google/gemma-2b-it" # O modelo base usado no treinamento
ADAPTER_HUB_ID = "peterson047/Tupi2LLM" # <<< SUBSTITUA PELO SEU HF REPO ID!!!
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
INSTRUCTION_TEXT = "o que significa:" # A mesma instrução usada no dataset

print(f"Modelo Base: {BASE_MODEL_ID}")
print(f"Adaptador LoRA: {ADAPTER_HUB_ID}")
print(f"Dispositivo: {DEVICE}")

Modelo Base: google/gemma-2b-it
Adaptador LoRA: peterson047/Tupi2LLM
Dispositivo: cuda


###Fazendo inferência

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

print(f"Carregando tokenizer de {BASE_MODEL_ID}...")  # ALTERADO AQUI
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)  # ALTERADO AQUI

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Pad token set to EOS token.")

print(f"Configurando Quantização (QLoRA)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # T4 suporta bfloat16
    bnb_4bit_use_double_quant=True,
)

print(f"Carregando modelo base ({BASE_MODEL_ID}) com quantização...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto", # Coloca na GPU automaticamente
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 # Define o dtype aqui também
)

print(f"Carregando e aplicando adaptadores LoRA de {ADAPTER_HUB_ID}...")
try:
    model = PeftModel.from_pretrained(model, ADAPTER_HUB_ID)
    print("Adaptadores LoRA carregados com sucesso.")
except Exception as e:
    print(f"Erro ao carregar adaptadores LoRA: {e}")
    print("Verifique se o caminho/ID do adaptador está correto e se os arquivos do adaptador existem no Hub.")
    # Você pode querer parar aqui se os adaptadores não carregarem
    # raise e

# Opcional: Mesclar adaptadores (pode consumir mais VRAM/RAM temporariamente)
# print("Mesclando adaptadores (opcional)...")
# try:
#    model = model.merge_and_unload()
#    print("Adaptadores mesclados.")
# except Exception as e:
#    print(f"Não foi possível mesclar adaptadores (pode ser falta de memória): {e}")
#    print("Continuando com o modelo não mesclado.")

model.eval() # Coloca em modo de avaliação
print(f"Modelo pronto para inferência no dispositivo: {model.device}")

Carregando tokenizer de google/gemma-2b-it...
Configurando Quantização (QLoRA)...
Carregando modelo base (google/gemma-2b-it) com quantização...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Carregando e aplicando adaptadores LoRA de peterson047/Tupi2LLM...
Erro ao carregar adaptadores LoRA: Can't find 'adapter_config.json' at 'peterson047/Tupi2LLM'
Verifique se o caminho/ID do adaptador está correto e se os arquivos do adaptador existem no Hub.
Modelo pronto para inferência no dispositivo: cuda:0


### Definindo a função de testes

In [ ]:
import torch
from transformers import GenerationConfig

def gerar_definicao(palavra_tupi, max_comprimento=150, temp=0.7, top_p_val=0.95):
    prompt = f"### Instruction:\n{INSTRUCTION_TEXT}\n\n### Input:\n{palavra_tupi}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True).to(DEVICE)

    print(f"\n--- Gerando para: {palavra_tupi} ---")

    # Configuração de geração
    generation_config = GenerationConfig(
        max_new_tokens=max_comprimento,
        temperature=temp,
        do_sample=True,
        top_k=50,
        top_p=top_p_val,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    with torch.no_grad(): # Desabilita cálculo de gradientes para inferência
        try:
            outputs = model.generate(**inputs, generation_config=generation_config)
            resultado_completo = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extrai apenas a resposta após "### Response:"
            partes = resultado_completo.split("### Response:")
            if len(partes) > 1:
                resposta = partes[-1].strip()
            else:
                # Se o marcador não foi gerado, tenta pegar o que veio depois do prompt original
                prompt_sem_resp = prompt.replace("### Response:\n", "")
                resposta = resultado_completo.replace(prompt_sem_resp, "").strip()

            print(f"Resposta Gerada:\n{resposta}")
            return resposta
        except Exception as e:
            print(f"Erro durante a geração: {e}")
            return f"Erro ao gerar: {e}"

print("Função gerar_definicao definida.")

Função gerar_definicao definida.


## Teste interativo

In [ ]:
# Teste com palavras do dicionário
palavras_teste = ["abá", "kunhã", "y", "oka", "îandé", "pirá", "morubixaba", "abá-eté"] # Adicione mais
for palavra in palavras_teste:
    gerar_definicao(palavra)


--- Gerando para: abá ---
Resposta Gerada:
o que significa é uma palavra que indica um estado ou ação.

--- Gerando para: kunhã ---
Resposta Gerada:
O que significa "kunhã"?

**O que significa "kunhã" é um verbo em português que significa "dar um passo ou fazer algo".**

--- Gerando para: y ---
Resposta Gerada:
y significa que significa "yes".

--- Gerando para: oka ---
Resposta Gerada:
O que significa é um verbo que indica um estado de coisas.

--- Gerando para: îandé ---
Resposta Gerada:
O que significa é?

--- Gerando para: pirá ---
Resposta Gerada:
O que significa "pirá"?

**Dica:** O termo "pirá" significa "um tipo de animal marítimo".

--- Gerando para: morubixaba ---
Resposta Gerada:
O que significa "morubixaba"?

--- Gerando para: abá-eté ---
Resposta Gerada:
a-ção.

**Definição:** A ação é uma ação física ou mental realizada por um ser vivo ou um objeto que transforma o seu estado de algo para outro.


In [ ]:

# Teste Interativo
print("\n--- Teste Interativo (digite 'sair' para terminar) ---")
while True:
    try:
        input_word = input("Digite uma palavra Tupi: ")
        if input_word.lower() == 'sair':
            break
        if input_word:
            gerar_definicao(input_word)
    except EOFError: # Para caso rode em ambientes que não suportam input direto
        print("EOF recebido, encerrando.")
        break
    except KeyboardInterrupt:
         print("\nInterrupção recebida, encerrando.")
         break


--- Teste Interativo (digite 'sair' para terminar) ---
Digite uma palavra Tupi: KABURE

--- Gerando para: KABURE ---
Resposta Gerada:
Kabura é um tipo de disertação que se faz sobre um tema específico.
Digite uma palavra Tupi: KAMB-0

--- Gerando para: KAMB-0 ---
Resposta Gerada:
Kambi está um animal do reino animal.
Digite uma palavra Tupi: carima, masa mole de mandioca, puba

--- Gerando para: carima, masa mole de mandioca, puba ---
Resposta Gerada:
O que significa "carima" é um termo que indica que algo é feito com maestria e cuidado.
Digite uma palavra Tupi: Qual a definição em português da palavra Tupi a seguir? KARU

--- Gerando para: Qual a definição em português da palavra Tupi a seguir? KARU ---
Resposta Gerada:
A palavra Tupi significa "um homem forte e corajoso".
Digite uma palavra Tupi: TAPI-UIA

--- Gerando para: TAPI-UIA ---
Resposta Gerada:
TAPI-UIA significa que o valor do parâmetro TAPI é igual a UIA.
Digite uma palavra Tupi: TERE-POMONGA

--- Gerando para: TERE-POMON